In [1]:
import pandas as pd

In [9]:
import os

In [10]:
os.chdir("/Users/macbook/Desktop/Projects/Analytics/Brazilian Ecommerce")

In [11]:
base_path = "/Users/macbook/Desktop/Projects/Analytics/Brazilian Ecommerce"

In [12]:
# List of all dataset files
files = {
    "categeory": "category_name_translation.csv",
    "customers": "customers.csv",
    "geolocation": "geolocation.csv",
    "order_items": "order_items.csv",
    "order_payments": "order_payments.csv",
    "order_reviews": "order_reviews.csv",
    "orders": "orders.csv",
    "products": "products.csv",
    "sellers": "sellers.csv"
}

In [13]:
# Load all datasets
dfs = {}
for name, filename in files.items():
    path = os.path.join(base_path, filename)
    try:
        df = pd.read_csv(path, encoding='utf-8')
        print(f"{name} loaded: {df.shape[0]} rows, {df.shape[1]} columns")
        dfs[name] = df
    except Exception as e:
        print(f"Error loading {filename}: {e}")

categeory loaded: 71 rows, 2 columns
customers loaded: 99441 rows, 5 columns
geolocation loaded: 1000163 rows, 5 columns
order_items loaded: 112650 rows, 7 columns
order_payments loaded: 103886 rows, 5 columns
order_reviews loaded: 99224 rows, 7 columns
orders loaded: 99441 rows, 8 columns
products loaded: 32951 rows, 9 columns
sellers loaded: 3095 rows, 4 columns


## Cleaning the `orders` Dataset (1)

Starting by cleaning `orders.csv` dataset first because I suspect it is the **central table** that connects to most of the other datasets in this project 



In [17]:
orders = dfs['orders']

In [19]:
orders.info()
orders.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [23]:
date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

In [27]:
orders[orders['order_purchase_timestamp'] > pd.Timestamp.now()].shape[0]

0

In [30]:
orders['order_id'].duplicated().sum()

0

In [32]:
orders[orders['order_id'].duplicated(keep=False)]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date


In [42]:
orders.columns.tolist()

['order_id',
 'customer_id',
 'order_status',
 'purchase_time',
 'order_approved_at',
 'order_delivered_carrier_date',
 'delivered_time',
 'order_estimated_delivery_date']

In [36]:
orders.columns = (
    orders.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

In [41]:
orders.rename(columns={
    'order_purchase_timestamp': 'purchase_time',
    'order_delivered_customer_date': 'delivered_time'
    
}, inplace=True)

In [44]:
orders.columns.tolist()

['order_id',
 'customer_id',
 'order_status',
 'purchase_time',
 'order_approved_at',
 'order_delivered_carrier_date',
 'delivered_time',
 'order_estimated_delivery_date']

In [46]:
orders.to_csv("orders_1.csv", index=False)

#### Cleaning Steps Applied:

1. **Loaded the dataset** from the original CSV file
2. **Checked for nulls** across all columns to identify incomplete data. I left the nulls as it is for now because there aren't much and I'll tackle them if need be on SQL during Data Exploration.
3. **Converted date fields** (`order_purchase_timestamp`, `order_approved_at`, etc.) to proper datetime format
4. **Checked for future timestamps** in `order_purchase_timestamp` - found none.
5. **Checked for duplicates in `order_id` - there were none.
6. **Standardized column names** by ensuring they are in lowercase and use snake_case formatting
7. **Kept all columns**, including those with missing values, for a complete record of all order types (delivered, canceled, etc.)
8. **Exported** the cleaned data to a new CSV file: `orders_1.csv`

## Cleaning the `order_reviews` Dataset (2)

Next, I'll clean the `order_reviews.csv` dataset, which contains customer-generated reviews for each order. This dataset provides critical insights into **customer satisfaction**, including review scores, written comments, and timestamps for when reviews were created and answered.

### 🧹 Cleaning Steps Planned:

1. **Load the dataset** and inspect column names and types  
2. **Check for missing values**, particularly in review titles and messages  
3. **Convert date columns** (`review_creation_date`, `review_answer_timestamp`) to proper datetime format  
4. **(Optional)** Create indicator columns for presence of review text  
5. **Ensure column naming consistency** using lowercase and snake_case  
6. **Verify uniqueness of `review_id`**  
7. **Export** the cleaned version to a new file, e.g., `order_reviews_1.csv`

In [49]:
orders_reviews = dfs['order_reviews']

In [50]:
orders_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [52]:
orders_reviews.isnull().sum()

review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

In [54]:
order_reviews = dfs['order_reviews']

In [56]:
order_reviews['review_creation_date'] = pd.to_datetime(order_reviews['review_creation_date'], errors='coerce')
order_reviews['review_answer_timestamp'] = pd.to_datetime(order_reviews['review_answer_timestamp'], errors='coerce')

In [61]:
# Optional: add indicator for message presence
order_reviews['has_message'] = order_reviews['review_comment_message'].notnull().astype(int)

In [63]:
order_reviews['has_message'].mean()  # % of reviews with text
order_reviews[order_reviews['has_message'] == 1]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,has_message
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,1
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53,1
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22,2018-05-23 16:45:47,1
12,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,NaN,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",2018-02-16,2018-02-20 10:52:22,1
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23,2018-05-24 03:00:01,1
...,...,...,...,...,...,...,...,...
99205,98fffa80dc9acbde7388bef1600f3b15,d398e9c82363c12527f71801bf0e6100,4,NaN,para este produto recebi de acordo com a compr...,2017-11-29,2017-11-30 15:52:51,1
99208,df5fae90e85354241d5d64a8955b2b09,509b86c65fe4e2ad5b96408cfef9755e,5,NaN,Entregou dentro do prazo. O produto chegou em ...,2018-02-07,2018-02-19 19:47:23,1
99215,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,3,NaN,"O produto não foi enviado com NF, não existe v...",2018-05-19,2018-05-20 21:51:06,1
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22,2018-03-23 09:10:43,1


In [65]:
order_reviews['review_id'].duplicated().sum()

814

In [67]:
order_reviews[order_reviews['review_id'].duplicated(keep=False)].sort_values('review_id')

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,has_message
46678,00130cbe1f9d422698c812ed8ded1919,dfcdfc43867d1c1381bfaf62d6b9c195,1,NaN,"O cartucho ""original HP"" 60XL não é reconhecid...",2018-03-07,2018-03-20 18:08:23,1
29841,00130cbe1f9d422698c812ed8ded1919,04a28263e085d399c97ae49e0b477efa,1,NaN,"O cartucho ""original HP"" 60XL não é reconhecid...",2018-03-07,2018-03-20 18:08:23,1
90677,0115633a9c298b6a98bcbe4eee75345f,78a4201f58af3463bdab842eea4bc801,5,NaN,NaN,2017-09-21,2017-09-26 03:27:47,0
63193,0115633a9c298b6a98bcbe4eee75345f,0c9850b2c179c1ef60d2855e2751d1fa,5,NaN,NaN,2017-09-21,2017-09-26 03:27:47,0
92876,0174caf0ee5964646040cd94e15ac95e,f93a732712407c02dce5dd5088d0f47b,1,NaN,Produto entregue dentro de embalagem do fornec...,2018-03-07,2018-03-08 03:00:53,1
...,...,...,...,...,...,...,...,...
31120,fe5c833752953fed3209646f1f63b53c,4863e15fa53273cc7219c58f5ffda4fb,1,NaN,"Comprei dois produtos e ambos, mesmo enviados ...",2018-02-28,2018-02-28 13:57:52,1
7870,ff2fc9e68f8aabfbe18d710b83aabd30,2da58e0a7dcfa4ce1e00fad9d03ca3b5,2,NaN,NaN,2018-03-17,2018-03-19 11:44:15,0
82521,ff2fc9e68f8aabfbe18d710b83aabd30,1078d496cc6ab9a8e6f2be77abf5091b,2,NaN,NaN,2018-03-17,2018-03-19 11:44:15,0
73951,ffb8cff872a625632ac983eb1f88843c,c44883fc2529b4aa03ca90e7e09d95b6,3,NaN,NaN,2017-07-22,2017-07-26 13:41:07,0


In [69]:
order_reviews['review_id'].value_counts().head(10)

7b606b0d57b078384f0b58eac1d41d78    3
dbdf1ea31790c8ecfcc6750525661a9b    3
32415bbf6e341d5d517080a796f79b5c    3
0c76e7a547a531e7bf9f0b99cba071c1    3
4219a80ab469e3fc9901437b73da3f75    3
abbfacb2964f74f6487c9c10ac46daa6    3
e44840754f12fad2b8646712121b349a    3
70509c441d994fa03d6c1457930c9024    3
2172867fd5b1a55f98fe4608e1547b4b    3
832acec9bbf4efe65c3fb6423d8b4ed7    3
Name: review_id, dtype: int64

In [71]:
order_reviews[order_reviews['review_id'] == '7b606b0d57b078384f0b58eac1d41d78']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,has_message
7500,7b606b0d57b078384f0b58eac1d41d78,f3028a8f41ea1ee2b461420913663f97,5,NaN,NaN,2017-02-15,2017-02-21 23:30:22,0
59859,7b606b0d57b078384f0b58eac1d41d78,2deb17060fc1ce18a85eba953ddcdeaf,5,NaN,NaN,2017-02-15,2017-02-21 23:30:22,0
61069,7b606b0d57b078384f0b58eac1d41d78,2f8f31eb2f7b6572836d662a6625c8e4,5,NaN,NaN,2017-02-15,2017-02-21 23:30:22,0


In [73]:
order_reviews['order_id'].duplicated().sum()

551

In [75]:
dupe_order_ids = order_reviews[order_reviews['order_id'].duplicated(keep=False)]
dupe_order_ids['order_id'].value_counts().head(1)

df56136b8031ecd28e200bb18e6ddb2e    3
Name: order_id, dtype: int64

In [77]:
order_reviews[order_reviews['order_id'] == 'df56136b8031ecd28e200bb18e6ddb2e']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,has_message
2952,c444278834184f72b1484dfe47de7f97,df56136b8031ecd28e200bb18e6ddb2e,5,NaN,NaN,2017-02-08,2017-02-14 13:58:48,0
13982,72a1098d5b410ae50fbc0509d26daeb9,df56136b8031ecd28e200bb18e6ddb2e,5,NaN,NaN,2017-02-07,2017-02-10 10:46:09,0
62728,44f3e54834d23c5570c1d010824d4d59,df56136b8031ecd28e200bb18e6ddb2e,5,NaN,NaN,2017-02-09,2017-02-09 09:07:28,0


In [79]:
order_reviews.drop_duplicates(subset='order_id', keep='first', inplace=True)

In [81]:
# Reload original reviews CSV to get full count again
original_reviews = pd.read_csv("order_reviews.csv")
print("Before drop:", original_reviews.shape[0])

# Current cleaned DataFrame
print("After drop:", order_reviews.shape[0])

Before drop: 99224
After drop: 98673


In [84]:
order_reviews.columns = (
    order_reviews.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

In [87]:
order_reviews.to_csv("order_reviews_1.csv", index=False)

In [88]:
orders_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98673 entries, 0 to 99223
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_id                98673 non-null  object        
 1   order_id                 98673 non-null  object        
 2   review_score             98673 non-null  int64         
 3   review_comment_title     11551 non-null  object        
 4   review_comment_message   40785 non-null  object        
 5   review_creation_date     98673 non-null  datetime64[ns]
 6   review_answer_timestamp  98673 non-null  datetime64[ns]
 7   has_message              98673 non-null  int64         
dtypes: datetime64[ns](2), int64(2), object(4)
memory usage: 6.8+ MB


#### Cleaning Steps Applied:

1. **Checked for missing values**  
   - Found that `review_comment_title` and `review_comment_message` had a large number of nulls.
   - These were retained, as they're optional and not critical for every review.

2. **Converted date columns to datetime format**  
   - `review_creation_date` and `review_answer_timestamp` were converted using `pd.to_datetime`.

3. **Added a binary flag for presence of message**  
   - Created a new column `has_message` to indicate whether a review includes a comment message.

4. **Detected and removed duplicate reviews for the same order**  
   - Found 551 `order_id`s with multiple reviews.
   - Since each order should only have one review, I kept only the first review per `order_id`.

5. **Standardized column names**  
   - Converted all column names to lowercase with underscores.

6. **Exported cleaned data**  
   - The cleaned dataset was saved as `order_reviews_1.csv`


## 🧹 Cleaning the `order_payments` Dataset (3)

Next, I'll clean the `order_payments` dataset to ensure it's ready for analysis.

This dataset contains payment-related information for each order, including:
- Payment type for example: credit card, boleto..
- Number of installments
- Amount paid

I'll start by inspecting the data structure, checking for null values, and identifying any duplicates or inconsistencies.

In [90]:
order_payments = dfs['order_payments']

In [92]:
order_payments.info()
order_payments.isnull().sum()
order_payments.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


#### Initial Observations:
- No missing values were found.
- Data types are appropriate.
- Some orders appear multiple times, reflecting multiple payments or installments.

#### Planned Cleaning Steps:
1. Check for duplicate rows.
2. Check for duplicated `order_id`s to understand multi-payment behavior.
3. Inspect unique values in `payment_type` for consistency.
4. Explore extreme or zero values in `payment_value`.
5. Standardize column names for consistency.


In [94]:
order_payments.duplicated().sum()
order_payments['order_id'].duplicated().sum()

4446

In [98]:
# Find order_ids that appear more than once
order_payments['order_id'].value_counts().loc[lambda x: x > 1].head(5)

fa65dad1b0e818e3ccc5cb0e39231352    29
ccf804e764ed5650cd8759557269dc13    26
285c2e15bebd4ac83635ccc563dc71f4    22
895ab968e7bb0d5659d16cd74cd1650c    21
fedcd9f7ccdc8cba3a18defedd1a5547    19
Name: order_id, dtype: int64

In [100]:
order_payments[order_payments['order_id'] == 'fa65dad1b0e818e3ccc5cb0e39231352']

,order_id,payment_sequential,payment_type,payment_installments,payment_value
4885,fa65dad1b0e818e3ccc5cb0e39231352,27,voucher,1,66.02
9985,fa65dad1b0e818e3ccc5cb0e39231352,4,voucher,1,29.16
14321,fa65dad1b0e818e3ccc5cb0e39231352,1,voucher,1,3.71
17274,fa65dad1b0e818e3ccc5cb0e39231352,9,voucher,1,1.08
19565,fa65dad1b0e818e3ccc5cb0e39231352,10,voucher,1,12.86
23074,fa65dad1b0e818e3ccc5cb0e39231352,2,voucher,1,8.51
24879,fa65dad1b0e818e3ccc5cb0e39231352,25,voucher,1,3.68
28330,fa65dad1b0e818e3ccc5cb0e39231352,5,voucher,1,0.66
29648,fa65dad1b0e818e3ccc5cb0e39231352,6,voucher,1,5.02
32519,fa65dad1b0e818e3ccc5cb0e39231352,11,voucher,1,4.03


In [102]:
order_payments['payment_type'].value_counts()

credit_card    76795
boleto         19784
voucher         5775
debit_card      1529
not_defined        3
Name: payment_type, dtype: int64

In [106]:
order_payments[order_payments['payment_type'] == 'not_defined']

,order_id,payment_sequential,payment_type,payment_installments,payment_value


In [ ]:
order_payments = order_payments[order_payments['payment_type'] != 'not_defined']

### Cleaning the `order_payments` Dataset

In this section, I cleaned the `order_payments` dataset by:

1. **Inspecting for duplicates**:
   - Found and confirmed multiple `order_id` entries are valid (installment payments).
   - Kept all rows, as each row represents part of a legitimate payment sequence.

2. **Verifying column values**:
   - Confirmed no null values exist.
   - Checked for unexpected `payment_type` values and found 3 rows labeled `'not_defined'` — all with a `payment_value` of 0.
   - These were likely invalid records and were dropped.

This dataset is now cleaned and ready for analysis or merging with others.

In [110]:
order_payments.to_csv("order_payments_1.csv", index=False)

## Cleaning the `order_items` Dataset (4)

Now clean the `order_items` dataset, which details individual items in each order. This dataset is crucial for analyzing product-level trends, pricing, and revenue calculations.

I'll begin by inspecting the dataset for:
- Missing values
- Data types and formatting
- Duplicate entries or inconsistencies

Let’s load the data and explore its structure.

In [33]:
order_items = dfs['order_items']
order_items.info()
order_items.isnull().sum()
order_items.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


🧹 Cleaning the order_items Dataset
I'll now clean the order_items dataset, which contains detailed information about individual items in customer orders. This table is crucial for understanding pricing, shipping timelines, and the relationship between products, sellers, and orders.

🛠 Steps to clean:

Convert shipping_limit_date to datetime for easier date comparisons.
Check for and remove duplicate rows if any.
(Optional) Rename or standardize column names (though current naming is consistent and clear).


In [34]:
order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date'], errors='coerce')

In [35]:
order_items.duplicated().sum()

0

In [36]:
order_items['order_id'].duplicated().sum()

13984

This simply means there are 13,984 orders with more than one item — not that there's something wrong.

Each row in order_items represents one item in an order. So repeated order_ids are expected and normal.

In [37]:
order_items['order_id'].value_counts().head(10)

8272b63d03f5f79c56e9e4120aec44ef    21
1b15974a0141d54e36626dca3fdc731a    20
ab14fdcfbe524636d65ee38360e22ce8    20
9ef13efd6949e4573a18964dd1bbe7f5    15
428a2f660dc84138d969ccd69a0ab6d5    15
9bdc4d4c71aa1de4606060929dee888c    14
73c8ab38f07dc94389065f7eba4f297a    14
37ee401157a3a0b28c9c6d0ed8c3b24b    13
2c2a19b5703863c908512d135aa6accc    12
c05d6a79e55da72ca780ce90364abed9    12
Name: order_id, dtype: int64

In [38]:
order_items['order_id'].value_counts().value_counts().sort_index()

1     88863
2      7516
3      1322
4       505
5       204
6       198
7        22
8         8
9         3
10        8
11        4
12        5
13        1
14        2
15        2
20        2
21        1
Name: order_id, dtype: int64

Most orders with a single item = 88,863
A small number had 2 to 5 items
A few had more than 10 items
The largest had 21 items

In [39]:
order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date'], errors='coerce')

In [40]:
order_items.isnull().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [41]:
order_items.to_csv("order_items_1.csv", index=False)

### Cleaning the `order_items` Dataset

I cleaned the `order_items` dataset, which contains details about individual items within each order. This dataset is crucial for analyzing product-level sales, pricing, and delivery commitments.

**Steps performed:**
- Confirmed the dataset had **112,650 rows** and **7 columns**
- Converted `shipping_limit_date` to datetime format for consistency
- Checked for and found **no missing values**
- Verified there were **no duplicate rows**
- Observed that some `order_id`s repeated — which was expected since multiple products can belong to a single order

The cleaned dataset was saved as `order_items_cleaned.csv`

## Cleaning the `Customers` Dataset (5)

I'll now clean the `customers` dataset, which contains customer IDs and location-related fields. This dataset is essential for connecting orders to customer regions and performing geographic analyses.

#### Steps in the cleaning process:
1. Inspect the structure and data types of the dataset.
2. Check for missing values and duplicates.
3. Standardize column names for consistency.
4. Verify data integrity (e.g., unique `customer_id`, valid ZIP codes).
5. Export the cleaned dataset.


In [16]:
customers = dfs["customers"]
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [18]:
customers['customer_id'].duplicated().sum()

0

In [20]:
customers.groupby('customer_unique_id')['customer_id'].nunique().value_counts().sort_index()

1     93099
2      2745
3       203
4        30
5         8
6         6
7         3
9         1
17        1
Name: customer_id, dtype: int64

In [22]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [24]:
customers.sample(10)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
33486,554c9763dcf0b0d75321961cf6c76d6c,cacb72306c8c51a817d1c7411fab5b7b,14740,viradouro,SP
83305,3411fc68c6539c73daf6c0e431297452,5c5280de1024ed0beca4f9939395730e,85877,sao miguel do iguacu,PR
61479,4498f5e3202243c58ae58247497599a5,7d5599666ef8e7e2fab7c3280e035705,46100,brumado,BA
54,8247b5583327ab8be19f96e1fb82f77b,d85547cd859833520b311b4458a14c1c,23970,parati,RJ
87757,5ce19e6c79a4a3d9de36c5cccf0eea3a,a87f3307c1b45456352b4d70632ecb05,65072,sao luis,MA
79383,fe8c2af8a8a466fb4131716fa9b3858a,6d2c0561abf4839164f41e365c17289c,19901,ourinhos,SP
65213,f729b7c28ff81599a20cea19508fe61f,a4a4ec2c68565e8d9815ff6e1cc13c8e,94450,viamao,RS
89193,3743de9608dba0325a5534fff7c367d6,3e5c928acf49c4b95e57af1f350d3493,69901,rio branco,AC
35377,af31ca29fb4190a0b56304428dbe90a9,75c60ae070be933925521fe45e739919,79570,aparecida do taboado,MS
60739,acf3b1b4ec4e2a6aa97acf15cfab1d5b,888bbf8be9eaceefedd820af2ddfb90f,24230,niteroi,RJ


In [26]:
customers[['customer_city', 'customer_state']].drop_duplicates().sort_values(by=['customer_state', 'customer_city']).reset_index(drop=True)

,customer_city,customer_state
0,brasileia,AC
1,cruzeiro do sul,AC
2,epitaciolandia,AC
3,manoel urbano,AC
4,porto acre,AC
...,...,...
4305,silvanopolis,TO
4306,sitio novo do tocantins,TO
4307,taguatinga,TO
4308,tocantinopolis,TO


In [28]:
customers['customer_state'].value_counts().sort_index()

AC       81
AL      413
AM      148
AP       68
BA     3380
CE     1336
DF     2140
ES     2033
GO     2020
MA      747
MG    11635
MS      715
MT      907
PA      975
PB      536
PE     1652
PI      495
PR     5045
RJ    12852
RN      485
RO      253
RR       46
RS     5466
SC     3637
SE      350
SP    41746
TO      280
Name: customer_state, dtype: int64

In [29]:
# Count of customers by state
state_counts = customers['customer_state'].value_counts()
print(state_counts)

SP    41746
RJ    12852
MG    11635
RS     5466
PR     5045
SC     3637
BA     3380
DF     2140
ES     2033
GO     2020
PE     1652
CE     1336
PA      975
MT      907
MA      747
MS      715
PB      536
PI      495
RN      485
AL      413
SE      350
TO      280
RO      253
AM      148
AC       81
AP       68
RR       46
Name: customer_state, dtype: int64


In [31]:
customers.to_csv("customers_1.csv", index=False)

### Cleaning the `customers` Dataset

I cleaned the `customers` dataset, which contains demographic and geographic information about the customers. This data helps identify unique customers, analyze regional distribution, and understand the customer base.

#### Steps performed:
- Confirmed the dataset had **99,441 rows** and **5 columns**
- Verified that all data types were appropriate (`int64` and `object`)
- Checked for and found **no missing values**
- Verified there were **no duplicate `customer_id` values**
- Investigated `customer_unique_id` and found expected repeats, indicating customers with multiple orders
- Explored the geographical spread by viewing distinct combinations of `customer_city` and `customer_state`

The cleaned dataset was saved as `customers_1.csv`.


## Cleaning the `geolocation` Dataset(6)

Next up is the `geolocation` dataset, which contains geographical data such as zip code prefixes, city names, state codes, and latitude/longitude coordinates. This dataset is useful for visualizing location distribution and potentially calculating distances for logistics analysis.

**Steps I'll follow:**
- Load and inspect the dataset
- Check the number of rows and columns
- Identify and handle any missing values
- Check for duplicate rows and duplicates in key fields like `geolocation_zip_code_prefix`
- Standardize column names if needed
- Assess data consistency (e.g., verify coordinate ranges)
- Export the cleaned dataset as `geolocation_1.csv`


In [43]:
geolocation = pd.read_csv("geolocation.csv")
geolocation.info()
geolocation.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [45]:
geolocation.isnull().sum()

geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64

In [47]:
geolocation.duplicated().sum()

261831

In [49]:
# Count how many times each row appears
dupes = geolocation.groupby(geolocation.columns.tolist()).size().reset_index(name='count')

# Sort by highest numbers
dupes = dupes.sort_values(by='count', ascending=False)

# Show the most top most ones
dupes.head(1)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,count
666533,88220,-27.102099,-48.629613,itapema,SC,314


In [51]:
# Filter the dataset to view all 314 matching rows
geolocation[
    (geolocation['geolocation_zip_code_prefix'] == 88220) &
    (geolocation['geolocation_lat'] == -27.102099) &
    (geolocation['geolocation_lng'] == -48.629613) &
    (geolocation['geolocation_city'] == 'itapema') &
    (geolocation['geolocation_state'] == 'SC')
]

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state


In [53]:
# Find all duplicate rows (entire row duplicates)
geo_duplicates = geolocation[geolocation.duplicated(keep=False)]

# View the top duplicate group
geo_duplicates.value_counts().head(1)

geolocation_zip_code_prefix  geolocation_lat  geolocation_lng  geolocation_city  geolocation_state
88220                        -27.102099       -48.629613       itapema           SC                   314
dtype: int64

In [55]:
geolocation[
    (geolocation['geolocation_zip_code_prefix'] == 88220) &
    (geolocation['geolocation_lat'] == -27.102099) &
    (geolocation['geolocation_lng'] == -48.629613) &
    (geolocation['geolocation_city'] == 'itapema') &
    (geolocation['geolocation_state'] == 'SC')
]

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state


In [57]:
geolocation[
    (geolocation['geolocation_zip_code_prefix'] == 88220) &
    (geolocation['geolocation_lat'].round(6) == -27.102099) &
    (geolocation['geolocation_lng'].round(6) == -48.629613) &
    (geolocation['geolocation_city'].str.strip().str.lower() == 'itapema') &
    (geolocation['geolocation_state'].str.strip().str.upper() == 'SC')
]

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
908079,88220,-27.102099,-48.629613,itapema,SC
908080,88220,-27.102099,-48.629613,itapema,SC
908081,88220,-27.102099,-48.629613,itapema,SC
908082,88220,-27.102099,-48.629613,itapema,SC
908092,88220,-27.102099,-48.629613,itapema,SC
...,...,...,...,...,...
909252,88220,-27.102099,-48.629613,itapema,SC
909270,88220,-27.102099,-48.629613,itapema,SC
909277,88220,-27.102099,-48.629613,itapema,SC
909286,88220,-27.102099,-48.629613,itapema,SC


In [59]:
geolocation = geolocation.drop_duplicates()

In [61]:
len(geolocation)

738332

In [63]:
geolocation.duplicated().sum()

0

In [65]:
geolocation.to_csv("geolocation_1.csv", index=False)

### Cleaning the `geolocation` Dataset

I cleaned the `geolocation` dataset, which contains zip code prefixes, coordinates, and location details of customers and sellers. This dataset is important for mapping and geographic clustering analyses.

#### Steps in the cleaning process:
1. Confirmed the dataset had **1,000,163 rows** and **5 columns**
2. Verified there were **no missing values**
3. Detected and dropped **261,831 exact duplicate rows**
4. Verified that the cleaned dataset now has **738,332 unique rows**
5. Saved the cleaned dataset as `geolocation_1.csv`

## Cleaning the products Dataset (7)

I'll now clean the products dataset, which includes product-specific information such as category, dimensions, and weight. This dataset is key for product-level analysis, including performance, pricing trends, and shipping characteristics.

#### Steps in the cleaning process:

Load the dataset and examine its structure and data types.
Check for missing values and duplicated rows.
Standardize and validate product categories and measurements.
Drop or impute nulls if necessary.
Export the cleaned dataset as products_1.csv.

In [91]:
products = pd.read_csv("products.csv")
products.info()
products.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [92]:
products['product_id'].is_unique

True

In [93]:
products['product_id'].duplicated().sum()

0

In [94]:
products = products.dropna(subset=['product_category_name'])

In [95]:
products.shape

(32341, 9)

In [96]:
products['product_category_name'].nunique()

73

In [97]:
products['product_category_name'].unique()

array(['perfumaria', 'artes', 'esporte_lazer', 'bebes',
       'utilidades_domesticas', 'instrumentos_musicais', 'cool_stuff',
       'moveis_decoracao', 'eletrodomesticos', 'brinquedos',
       'cama_mesa_banho', 'construcao_ferramentas_seguranca',
       'informatica_acessorios', 'beleza_saude', 'malas_acessorios',
       'ferramentas_jardim', 'moveis_escritorio', 'automotivo',
       'eletronicos', 'fashion_calcados', 'telefonia', 'papelaria',
       'fashion_bolsas_e_acessorios', 'pcs', 'casa_construcao',
       'relogios_presentes', 'construcao_ferramentas_construcao',
       'pet_shop', 'eletroportateis', 'agro_industria_e_comercio',
       'moveis_sala', 'sinalizacao_e_seguranca', 'climatizacao',
       'consoles_games', 'livros_interesse_geral',
       'construcao_ferramentas_ferramentas',
       'fashion_underwear_e_moda_praia', 'fashion_roupa_masculina',
       'moveis_cozinha_area_de_servico_jantar_e_jardim',
       'industria_comercio_e_negocios', 'telefonia_fixa',
       '

In [98]:
sorted(products['product_category_name'].unique())

['agro_industria_e_comercio',
 'alimentos',
 'alimentos_bebidas',
 'artes',
 'artes_e_artesanato',
 'artigos_de_festas',
 'artigos_de_natal',
 'audio',
 'automotivo',
 'bebes',
 'bebidas',
 'beleza_saude',
 'brinquedos',
 'cama_mesa_banho',
 'casa_conforto',
 'casa_conforto_2',
 'casa_construcao',
 'cds_dvds_musicais',
 'cine_foto',
 'climatizacao',
 'consoles_games',
 'construcao_ferramentas_construcao',
 'construcao_ferramentas_ferramentas',
 'construcao_ferramentas_iluminacao',
 'construcao_ferramentas_jardim',
 'construcao_ferramentas_seguranca',
 'cool_stuff',
 'dvds_blu_ray',
 'eletrodomesticos',
 'eletrodomesticos_2',
 'eletronicos',
 'eletroportateis',
 'esporte_lazer',
 'fashion_bolsas_e_acessorios',
 'fashion_calcados',
 'fashion_esporte',
 'fashion_roupa_feminina',
 'fashion_roupa_infanto_juvenil',
 'fashion_roupa_masculina',
 'fashion_underwear_e_moda_praia',
 'ferramentas_jardim',
 'flores',
 'fraldas_higiene',
 'industria_comercio_e_negocios',
 'informatica_acessorios',
 

In [99]:
products[['product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']].describe()

,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32340.000000,32340.000000,32340.000000,32340.000000
mean,2276.956586,30.854545,16.958813,23.208596
std,4279.291845,16.955965,13.636115,12.078762
min,0.000000,7.000000,2.000000,6.000000
25%,300.000000,18.000000,8.000000,15.000000
50%,700.000000,25.000000,13.000000,20.000000
75%,1900.000000,38.000000,21.000000,30.000000
max,40425.000000,105.000000,105.000000,118.000000


In [100]:
products = products[products['product_weight_g'] > 0]

In [101]:
(products['product_weight_g'] == 0).sum()

0

In [102]:
products.to_csv("products_1.csv", index=False)

### Cleaning the `Products` Dataset (7)

I cleaned the product` dataset, which provides detailed information about each product sold, including its category, physical attributes, and descriptions. This dataset is useful for product-level analysis and understanding assortment characteristics.

#### Steps in the cleaning process:
1. Loaded the dataset with **32,951 rows** and **9 columns**
2. Checked and confirmed that `product_id` values are **all unique**
3. Identified **missing values** in several columns, especially in `product_category_name`
4. Dropped rows where `product_category_name` was missing, reducing the dataset to **32,341 rows**
5. Verified that the remaining float columns were appropriate for numeric analysis
6. Checked for product records with zero weight — confirmed that none exist in the filtered dataset
7. Also checked how many categories of products are there: **73**

The cleaned dataset was saved as `products_1.csv`

## Cleaning the sellers Dataset (8)

I'll now clean the sellers dataset, which contains information about each seller, including their location. This dataset is essential for analyzing the geographic distribution of sellers and linking them to their sales performance.

#### Steps in the cleaning process:

Load the dataset and inspect its structure.
Check for missing values or duplicates.
Validate seller ID uniqueness.
Standardize and clean location fields if necessary.
Export the cleaned dataset as sellers_1.csv.

In [104]:
sellers = pd.read_csv("sellers.csv")
sellers.info()
sellers.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [106]:
sellers.duplicated().sum()

0

In [108]:
sellers['seller_id'].duplicated().sum()

0

In [110]:
sellers['seller_state'].value_counts()

SP    1849
PR     349
MG     244
SC     190
RJ     171
RS     129
GO      40
DF      30
ES      23
BA      19
CE      13
PE       9
PB       6
RN       5
MS       5
MT       4
RO       2
SE       2
PI       1
AC       1
MA       1
AM       1
PA       1
Name: seller_state, dtype: int64

In [112]:
sellers.to_csv("sellers_1.csv", index=False)

### Cleaning the `sellers` Dataset (8)

I cleaned the `sellers` dataset, which included seller identifiers and their location details. This dataset was essential for mapping seller distribution and linking them to order transactions.

### Steps in the cleaning process:
1. Confirmed the dataset contained **3,095 rows** and **4 columns**.
2. Verified there were **no missing values**.
3. Checked for and found **no duplicate rows**.
4. Confirmed that all `seller_id` values were **unique**.
5. Examined the `seller_state` column and observed that sellers were spread across **23 states**.

The cleaned dataset was saved as `sellers_1.csv`.


### Cleaning the `product_category_name_translation` Dataset (9)

Final Dataset next: `product_category_name_translation`, which maps Portuguese product category names to their English equivalents. This dataset is important for making product category analysis more interpretable for non-Portuguese speakers.

#### Steps in the cleaning process:
1. Load and inspect the structure and data types.
2. Check for missing values and duplicates.
3. Standardize column names if necessary.
4. Verify all Portuguese category names exist in the products dataset.
5. Export the cleaned dataset.

In [115]:
category_translation = pd.read_csv("category_name_translation.csv")
category_translation.info()
category_translation.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [119]:
category_translation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [123]:
category_translation.info()
category_translation.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [124]:
# Check for missing values
category_translation.isna().sum()

product_category_name            0
product_category_name_english    0
dtype: int64

In [125]:
# Check for duplicate rows
category_translation.duplicated().sum()

0

In [126]:
# Check for duplicate values in each column
category_translation['product_category_name'].duplicated().sum()
category_translation['product_category_name_english'].duplicated().sum()

0

In [128]:
category_translation.to_csv('category_name_translation_1.csv', index=False)

###  Cleaning the category_translation Dataset (8)

I cleaned the category_translation dataset, which maps Portuguese product category names to their English equivalents. This dataset is useful for making product data more interpretable for non-Portuguese speakers.

#### Steps in the cleaning process:
Loaded the dataset with 71 rows and 2 columns
Verified there were no missing values
Checked for duplicate rows and duplicate category names — none were found
Renamed and saved the cleaned dataset as category_name_translation_1.csv

# Cleaning Project End Summary

I have completed the data cleaning process for the Olist e-commerce dataset, which included 9 different datasets:

orders
order_items
order_payments
order_reviews
customers
geolocation
products
sellers
category_translation
Each dataset was inspected, cleaned for missing or duplicate values, and exported with a _1 suffix in the filename to indicate the cleaned version (e.g., geolocation_1.csv).